<!--<badge>--><a href="https://colab.research.google.com/github/ankur-98/BERT_GLUE/blob/main/multi_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# For colab run:

In [1]:
# Switch to GPU runtime
! git clone https://github.com/ankur-98/BERT_GLUE.git
import os 
os.chdir("BERT_GLUE")
! pip install datasets transformers

Cloning into 'BERT_GLUE'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 107 (delta 53), reused 46 (delta 21), pack-reused 0
Receiving objects: 100% (107/107), 32.42 KiB | 310.00 KiB/s, done.
Resolving deltas: 100% (53/53), done.
     |████████████████████████████████| 225kB 4.3MB/s 
     |████████████████████████████████| 2.3MB 31.4MB/s 
     |████████████████████████████████| 245kB 35.8MB/s 
     |████████████████████████████████| 112kB 36.8MB/s 
     |████████████████████████████████| 3.3MB 33.6MB/s 
     |████████████████████████████████| 901kB 34.7MB/s 


# Imports

In [1]:
import torch
from tqdm.auto import tqdm
from dataloader import get_dataloader
from transformers import BertModel
from model import BERTClassifierModel
from train import training_step, eval_step
from util import *

# Configs
### Tasks: {"cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"}

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_checkpoint="bert-base-uncased"
tasks = ["cola","sst2"]
batch_size=64
steps = 200
lr = 2e-5
lr_scheduler_type = "linear" # "linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"

# Load Dataloader and Pre-trained BERT Model

In [3]:
num_labels = [3 if task.startswith("mnli") else 1 if task=="stsb" else 2 for task in tasks]
train_epoch_iterator = [get_dataloader(task, model_checkpoint, "train", batch_size=batch_size) for task in tasks]
eval_epoch_iterator = [get_dataloader(task, model_checkpoint, "validation", batch_size=batch_size) for task in tasks]
BERT_model = BertModel.from_pretrained(model_checkpoint)
models = [BERTClassifierModel(BERT_model, num_labels=num_label, task=task).to(device) for num_label, task in zip(num_labels, tasks)]


Reusing dataset glue (C:\Users\ankur\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 2/2 [00:00<00:00, 36.77ba/s]


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 1063
    })
})


Reusing dataset glue (C:\Users\ankur\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 2/2 [00:00<00:00, 19.45ba/s]


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 872
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 1821
    })
})


Reusing dataset glue (C:\Users\ankur\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 2/2 [00:00<00:00, 36.85ba/s]


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 1063
    })
})


Reusing dataset glue (C:\Users\ankur\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 2/2 [00:00<00:00, 21.74ba/s]


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 872
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 1821
    })
})


# Optimizer and LR Scheduler

In [4]:
Optimizers = [create_optimizer(model, learning_rate=lr) for model in models]
LR_schedulers = [create_scheduler(Optimizer, lr_scheduler_type) for Optimizer in Optimizers]
Metrics, Metrics_1 = zip(*[get_metrics(task) for task in tasks])
tr_loss = []
eval_loss = []
tr_metrics = [[] for task in tasks]
eval_metrics = [[] for task in tasks]
tr_metrics_1 = [[] for task in tasks]
eval_metrics_1 = [[] for task in tasks]

Couldn't find file locally at matthews_correlation\matthews_correlation.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.6.2/metrics/matthews_correlation/matthews_correlation.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/metrics/matthews_correlation/matthews_correlation.py.


# Training loop

In [5]:
global_steps = 0
max_step_in_dataloaders = max([len(iterator) for iterator in train_epoch_iterator])
trange = range(max_step_in_dataloaders)
loss_pbar = tqdm(trange, initial=global_steps, total=steps, desc=f"step summary:")
pbars = [tqdm(trange, initial=global_steps, total=steps, desc=f"{task}") for task in tasks]
for e in range((steps//max_step_in_dataloaders)+1):

    for step in trange:
        global_steps += 1
        step_loss = 0
        for i, task in enumerate(tasks):
            iterator = iter(train_epoch_iterator[i])
            pbars[i].update()
            
            inputs = prepare_inputs(iterator.next(), device)
            step_task_loss, step_metric, step_metric_1 = training_step(models[i], inputs, 
                                                                  Optimizers[i], LR_schedulers[i], 
                                                                  Metrics[i], Metrics_1[i])

            step_loss += step_task_loss
            tr_metrics[i].append(torch.tensor(list(step_metric.values())[0]))
            if Metrics_1[i] is not None: tr_metrics_1[i].append(torch.tensor(list(step_metric_1.values())[0]))
            
            step_evaluation = {}
            step_evaluation[f"{Metrics[i].__class__.__name__}"] = torch.stack(tr_metrics[i])[-len(train_epoch_iterator[i]):].mean().item()
            if Metrics_1[i] is not None:
                step_evaluation[f"{Metrics_1[i].__class__.__name__}"] = torch.stack(tr_metrics_1[i])[-len(train_epoch_iterator[i]):].mean().item()
            pbars[i].set_postfix(step_evaluation)
        loss_pbar.update()
        tr_loss.append(step_loss)
        step_evaluation = {}
        step_evaluation['loss'] = torch.stack(tr_loss[-len(train_epoch_iterator[0]):]).mean().item()
        loss_pbar.set_postfix(step_evaluation)
        
        if global_steps == steps:
            break

step summary::   0%|          | 0/200 [00:00<?, ?it/s]


step summary::   0%|          | 1/200 [00:03<11:31,  3.47s/it, loss=1.36]

step summary::   1%|          | 2/200 [00:06<10:34,  3.21s/it, loss=1.3] 

step summary::   2%|▏         | 3/200 [00:08<09:51,  3.00s/it, loss=1.25]

step summary::   2%|▏         | 4/200 [00:11<09:19,  2.86s/it, loss=1.2] 

step summary::   2%|▎         | 5/200 [00:13<09:00,  2.77s/it, loss=1.17]

step summary::   3%|▎         | 6/200 [00:16<08:44,  2.71s/it, loss=1.14]

step summary::   4%|▎         | 7/200 [00:18<08:32,  2.66s/it, loss=1.12]

step summary::   4%|▍         | 8/200 [00:21<08:23,  2.62s/it, loss=1.09]

step summary::   4%|▍         | 9/200 [00:24<08:17,  2.61s/it, loss=1.06]

step summary::   5%|▌         | 10/200 [00:26<08:11,  2.59s/it, loss=1.04]

step summary::   6%|▌         | 11/200 [00:29<08:15,  2.62s/it, loss=1.02]

step summary::   6%|▌         | 12/200 [00:31<08:10,  2.61s/it, loss=1.01]

step summary::   6%|▋         | 13/200 [

In [6]:
for i, task in enumerate(tasks):
    trange = range(len(eval_epoch_iterator[i]))
    loss_pbar = tqdm(trange, desc=f"step summary:")
    pbars = [tqdm(trange, desc=f"{task}") for task in tasks]
    for step in trange:
        step_loss = 0
        iterator = iter(eval_epoch_iterator[i])
        pbars[i].update()
        
        inputs = prepare_inputs(iterator.next(), device)
        step_task_loss, step_metric, step_metric_1 = eval_step(models[i], inputs, 
                                                               Metrics[i], Metrics_1[i])

        step_loss += step_task_loss
        eval_metrics[i].append(torch.tensor(list(step_metric.values())[0]))
        if Metrics_1[i] is not None: eval_metrics_1[i].append(torch.tensor(list(step_metric_1.values())[0]))
        
        step_evaluation = {}
        step_evaluation[f"{Metrics[i].__class__.__name__}"] = torch.stack(eval_metrics[i]).mean().item()
        if Metrics_1[i] is not None:
            step_evaluation[f"{Metrics_1[i].__class__.__name__}"] = torch.stack(eval_metrics_1[i]).mean().item()
        pbars[i].set_postfix(step_evaluation)
    loss_pbar.update()
    eval_loss.append(step_loss)
    step_evaluation = {}
    step_evaluation['loss'] = torch.stack(eval_loss).mean().item()
    loss_pbar.set_postfix(step_evaluation)



cola:   0%|          | 0/16 [00:00<?, ?it/s]


cola: 100%|██████████| 16/16 [00:08<00:00,  2.06it/s, MatthewsCorelation=0]



step summary::   6%|▋         | 1/16 [00:08<02:04,  8.28s/it, loss=0.659]



cola: 100%|██████████| 16/16 [00:08<00:00,  1.93it/s, MatthewsCorelation=0]




















































In [ ]:
print(f"Evaluation begins in batches of {batch_size}..")
global_steps = 0
trange = range(len(eval_epoch_iterator))
pbar_eval = tqdm(trange)
iterator = iter(eval_epoch_iterator)
for step in trange:
    global_steps += 1
    pbar_eval.update()
    
    inputs = prepare_inputs(iterator.next(), device)
    step_loss, step_metric, step_metric_1 = eval_step(model, inputs, Metric, Metric_1)
    eval_loss += step_loss
    eval_metric.append(torch.tensor(list(step_metric.values())[0]))
    if Metric_1 is not None: eval_metric_1.append(torch.tensor(list(step_metric_1.values())[0]))
    
    step_evaluation = {}
    step_evaluation['loss'] = (eval_loss/global_steps).item()
    step_evaluation[f"{Metric.__class__.__name__}"] = torch.stack(eval_metric).mean().item()
    if Metric_1 is not None:
        step_evaluation[f"{Metric_1.__class__.__name__}"] = torch.stack(eval_metric_1).mean().item()
    pbar_eval.set_postfix(step_evaluation)
    
    if global_steps == steps: 
        break